# Design Project 2

This notebook makes use of [SymPy](https://docs.sympy.org/), which is a Python library for symbolic mathematics.

In [144]:
import sympy as sym
import numpy as np
from sympy.physics import mechanics
from sympy import *
from scipy import linalg

## Equation of Motion Derivation

Most of this derivation comes directly from Dr. Bretl's repository https://github.com/tbretl/ae353-sp22/tree/main/projects/02_segbot

Define physical constants, consistent with the URDF file:

In [95]:
# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

# Station parameters
station_velocity = -0.5 # <-- FIXME (change the velocity to change gravity)
station_radius = 20.    # <-- radius in meters of inside surface of station

# Acceleration of artifical gravity
g = station_velocity**2 * station_radius

Convert all physical constants to rational numbers:

In [96]:
# Dimensions
h = sym.nsimplify(h)
a = sym.nsimplify(a)
r = sym.nsimplify(r)

# Masses
mb = sym.nsimplify(mb)
mw = sym.nsimplify(mw)
m = sym.nsimplify(m)

# MOIs
Jx = sym.nsimplify(Jx)
Jy = sym.nsimplify(Jy)
Jz = sym.nsimplify(Jz)
Jw = sym.nsimplify(Jw)
Jwt = sym.nsimplify(Jwt)

# Gravity
g = sym.nsimplify(g)

Define variables (with a flag to indicate they are all real numbers — this helps simplify):

In [97]:
(e_l,
 e_h,
 v1,
 v2,
 w,
 theta,
 tau_R,
 tau_L) = sym.symbols('e_l, e_h, v1, v2, w, theta, tau_R, tau_L', real=True)

Compute equations of motion, excluding lateral and heading errors (see [Tuttle, 2014](https://docs.lib.purdue.edu/cgi/viewcontent.cgi?article=1415&context=open_access_theses)):

In [110]:
M = sym.Matrix([[m + 2 * Jw / r**2, 0, mb * h * sym.cos(theta)],
                [0, (Jx + mb * h**2) * sym.sin(theta)**2 + Jz * sym.cos(theta)**2 + (2 * Jw * a**2 / r**2) + 2 * mw * a**2, 0],
                [mb * h * sym.cos(theta), 0, Jy * mb * h**2]])
N = sym.Matrix([[mb * h * (w**2 + v2**2) * sym.sin(theta)],
                [-2 * (Jx - Jz + m * h**2) * sym.cos(theta) * sym.sin(theta) * w * v2 - mb * h * sym.sin(theta) * v1 * w],
                [(Jx - Jz + mb * h**2) * sym.cos(theta) * sym.sin(theta) * w**2 + mb * g * h * sym.sin(theta)]])
R = sym.Matrix([[1 / r, 1 / r],
                [-a / r, a / r],
                [-1, -1]])
f1 = sym.simplify(M.inv() * (N + R * sym.Matrix([[tau_L], [tau_R]])))


In [111]:
M

Matrix([
[           78/5,                                                            0, 18*cos(theta)/5],
[              0, 4289*sin(theta)**2/2000 + 1169*cos(theta)**2/2000 + 441/1000,               0],
[18*cos(theta)/5,                                                            0,         108/125]])

In [112]:
N

Matrix([
[                       (18*v2**2/5 + 18*w**2/5)*sin(theta)],
[-18*v1*w*sin(theta)/5 - 444*v2*w*sin(theta)*cos(theta)/125],
[          39*w**2*sin(theta)*cos(theta)/25 + 18*sin(theta)]])

In [113]:
R

Matrix([
[ 40/13, 40/13],
[-14/13, 14/13],
[    -1,    -1]])

Compute full equations of motion:

In [114]:
f = sym.Matrix([[v1 * sym.sin(e_h)],
                [w],
                [f1]])

Display the vector-valued function $f$ for which the equations of motion can be written as

$$\begin{bmatrix} \dot{e}_\text{lateral} \\ \dot{e}_\text{heading} \\ \dot{v} \\ \dot{w} \\ \ddot{\theta} \end{bmatrix} = f(e_\text{lateral}, e_\text{heading}, v, w, \theta, \dot{\theta}, \tau_R, \tau_L)$$

In [115]:
f

Matrix([
[                                                                                                                                                                    v1*sin(e_h)],
[                                                                                                                                                                              w],
[        -(1200*tau_L + 1200*tau_R + 1404*(v2**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                   32*(-875*tau_L + 875*tau_R - 2925*v1*w*sin(theta) - 1443*v2*w*sin(2*theta))/(13*(3120*sin(theta)**2 + 2051))],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*tau_L + 100*tau_R + 117*(v2**2 + w**2)*sin(theta))*cos(theta) - 76050*sin(theta))/(702*(25*cos(theta)**2 - 26))]])

Note that one of these ODEs is second-order — you will have to replace this with a set of two first-order ODEs, as usual.

## Linearization

We will linearize the system using a state-space model.

Our state is defined,
$$x = \left[\begin{matrix}e_{lat} - e_{lat,e}\\e_{head} - e_{head,e}\\v_1 - v_{1,e}\\w - w_e\\v_2 - v_{2,e}\\ \theta - \theta_e\end{matrix}\right]$$
where
$$\dot{\theta} = v_2$$


To convert the model to a state-space format, $\dot{x} = Ax +Bu$, we must solve for the A & B matrices.

In [139]:
# add theta_dot to the state matrix

f = sym.Matrix([[v1 * sym.sin(e_h)],
                [w],
                [f1],
                [v2]])

u = sym.Matrix([[tau_R], 
                [tau_L]])
# print(latex(f))

For $\dot{x}$ to equal 0, the equilibrium point must be defined as below.

In [105]:
e_le = 0.                         # error = 0 at equilibrium
e_he = 0.                         # error = 0 at equilibrium
v1e = 1.                           # constant velocity (m/s)
v2e = 0.                           # rad/s (theta_dot)
we = 0.                            # rad/s
thetae = 0.                        # rad
tau_Re = 0.                        # Nm
tau_Le = 0.                        # Nm

In [138]:
def linearize_model(f, u, e_le, e_he, v1e, v2e, we, thetae, tau_Re, tau_Le) :
    f_num = sym.lambdify([e_l, e_h, v1, v2, w, theta, tau_R, tau_L], f)
    
    # verify equilibrium point
    eq = f_num(e_le, e_he, v1e, v2e, we, thetae, tau_Re, tau_Le)
    #print(np.allclose(eq, 0.))
    if not np.allclose(f_num(e_le, e_he, v1e, v2e, we, thetae, tau_Re, tau_Le), 0.):
        raise Exception('equilibrium point is invalid')
        
    # Calculate symbolic A & B
    A_sym = f.jacobian([e_l, e_h, v1, v2, w, theta])
    B_sym = f.jacobian([tau_R, tau_L])
    
    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([e_l, e_h, v1, v2, w, theta, tau_R, tau_L], A_sym)
    B_num = sym.lambdify([e_l, e_h, v1, v2, w, theta, tau_R, tau_L], B_sym)
    
    # Find A and B in numeric form
    A = A_num(e_le, e_he, v1e, v2e, we, thetae, tau_Re, tau_Le).astype(float)
    B = B_num(e_le, e_he, v1e, v2e, we, thetae, tau_Re, tau_Le).astype(float)
    
    return A, B
    

A, B = linearize_model(f, u, e_le, e_he, v1e, v2e, we, thetae, tau_Re, tau_Le)
A = sym.Matrix(A)
B = sym.Matrix(B)

# print("A: ")
# print(latex(A))
# print("B: ")
# print(latex(B))

## Controllability

In [155]:
A = (np.array(A)).astype(float)
B = np.array(B).astype(float)

W = B
for i in range(1, len(A)):
        col = np.linalg.matrix_power(A, i) @ B 
        W = np.block([W, col])

In [156]:
# Check size of controllability matrix
print(f'shape of W is {W.shape}\n')

# Find determinant of controllability matrix
print(f'det(W) = {linalg.det(W)}\n')

# Find rank of controllability matrix
print(f'rank(W) = {np.linalg.matrix_rank(W)}')
print(f'number of states is {A.shape[0]}')

shape of W is (6, 12)



ValueError: expected square matrix